In [1]:
# Bibliotecas
import sys
sys.path.append('/home/jp/.snap/snap-python')
import snappy
from snappy import GPF
from snappy import ProductIO
from snappy import HashMap
from snappy import jpy
from snappy import ProductUtils
from snappy import WKTReader
import rasterio as rio
import matplotlib.pyplot as plt

INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


In [2]:
base_path = '/media/jp/FreeAgent GoFlex Drive/SAR/SLC_raw/AE04/'
file_path = 'S1A_IW_SLC__1SDV_20220817T090719_20220817T090746_044590_055287_A29D.zip'
file = base_path + file_path

In [3]:
# Funçao para entender os parâmetros de cada operador
def op_info(operator):
  op = snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)
  print('Nome do operador: {} '.format(op.getOperatorDescriptor().getName()))
  print('Nome do operador na API: {} '.format(op.getOperatorDescriptor().getAlias()))
  param_desc = op.getOperatorDescriptor().getParameterDescriptors()
  for i in param_desc:
    print('{}: {}\nValor padrao: {}\n'.format(i.getName(), i.getDescription(), i.getDefaultValue()))

In [4]:
# Acessando os operadores Java
HashMap = jpy.get_type('java.util.HashMap')
parameters = HashMap()

# Função para ler arquivo
def read(file):
  return ProductIO.readProduct(file)

# Funçao para gravar arquivo
def write(product, file):
  ProductIO.writeProduct(product, file, "GeoTIFF")

In [5]:
# TOPSAR-Split
def split(product, sub, first_burst, last_burst):
    parameters.put('subswath', sub)
    parameters.put('selectedPolarisations', 'VH,VV')
    parameters.put('firstBurstIndex', first_burst)
    parameters.put('lastBurstIndex', last_burst)
    return GPF.createProduct("TOPSAR-Split", parameters, product)

In [6]:
# Orbit files
def apply_orbit_file(product):   
    parameters.put("Orbit State Vectors", "Sentinel Precise (Auto Download)")
    parameters.put("Polynomial Degree", 3) 
    parameters.put('continueOnFail', 'false')   
    return GPF.createProduct("Apply-Orbit-File", parameters, product)

In [7]:
# Calibration
def calibration(product):
    parameters.put('selectedPolarisations', 'VH,VV')
    parameters.put('outputImageInComplex', 'true')
    parameters.put('outputImageScaleInDb', 'false')

    return GPF.createProduct('Calibration', parameters, product)

In [8]:
# Deburst
def deburst(product):
    parameters.put("Polarisations", "VH, VV")
    return GPF.createProduct("TOPSAR-Deburst", parameters, product)

In [9]:
# Matriz C2
def C2_matrix(product):
  parameters.put('matrix', 'C2')

  return GPF.createProduct('Polarimetric-Matrices', parameters, product)

In [10]:
# Speckle filter
def speckle_filter(product):
  return GPF.createProduct('Speckle-Filter', parameters, product)

In [11]:
# Terrain Correction
def terrain_correction(product):
  parameters.put('demName', 'Copernicus 30m Global DEM')
  parameters.put('demResamplingMethod', 'NEAREST_NEIGHBOUR')
  parameters.put('imgResamplingMethod', 'NEAREST_NEIGHBOUR')
  parameters.put('mapProjection', 'EPSG:32722')
  parameters.put('pixelSpacingInMeter', 14.78)
  parameters.put('outputComplex', 'true')
  return GPF.createProduct('Terrain-Correction', parameters, product)

In [12]:
# Multilooking
def multilooking(product):
  parameters.put('nRgLooks', 4)
  parameters.put('nAzLooks', 1)
  parameters.put('outputIntensity', 'true')
  parameters.put('grSquarePixel', 'true')
  return GPF.createProduct('Multilook', parameters, product )

In [13]:
# Linear to dB
def linear_to_db(product):
  #parameters.put('sourceBands', 'Intensity_IW2_VH, Intensity_IW2_VV')
  return GPF.createProduct('LinearToFromdB', parameters, product)

In [14]:
def subset(product, wkt):
    geom = WKTReader().read(wkt)
    parameters.put('copyMetadata', True)
    parameters.put('geoRegion', geom)
    SAR_image_subset = GPF.createProduct('Subset', parameters, product)
    return SAR_image_subset

In [15]:
# # Fluxo de processos
# def workflow(file):

#   print('read')
#   product = read(file)

#   print('TOPSAR-split')
#   spt = split(product)

#   print('Orbit file')
#   orbit = apply_orbit_file(spt)

#   print('Calibration')
#   cal = calibration(orbit)

#   print('Deburst')
#   deb = deburst(cal)

#   print('C2')
#   C2 = C2_matrix(deb)
#   write(C2, '/home/jp/Documents/IME/TCC/CODE/Capstone-Project/data/c2/area4_'+'C2.tif')

#   # print('Speckle filter')
#   spk = speckle_filter(deb)
#   spk = deb

#   print('Multilooking')
#   mult = multilooking(spk)

#   print('Terrain Correction')
#   TC = terrain_correction(mult)

#   print('to dB')
#   dB = linear_to_db(TC)
#   write(dB, '/media/jp/FreeAgent GoFlex Drive/SAR/SLC_raw/AE04/area4_split_orb_cal_deb_spk_mult_TC_dB.tif')


In [16]:
wkt_dict = {
    'area4': 'POLYGON((-50.769733 -4.627111, -50.562011 -4.627011, -50.561858 -4.865567, -50.769656 -4.865669, -50.769733 -4.627111))'
}

swath_dict = {
    'area4': {
        'sub': 'IW1',
        'first': '1',
        'last': '2'
    } 
}

In [17]:
# Fluxo de processos
def workflow_SLC(file, area_name):

  print('read')
  product = read(file)

  print('TOPSAR-split')
  swath = swath_dict.get(area_name)
  spt = split(product, swath.get('sub'), swath.get('first'), swath.get('last'))

  print('Orbit file')
  orbit = apply_orbit_file(spt)

  print('Calibration')
  cal = calibration(orbit)

  print('Deburst')
  deb = deburst(cal)

  print('Terrain Correction')
  TC = terrain_correction(deb)

  TC = subset(TC, wkt_dict.get(area_name))

  print('C2')
  C2 = C2_matrix(TC)
  write(C2, f'/home/jp/Documents/IME/TCC/CODE/Capstone-Project/data/c2/{area_name}_'+'C2.tif')

  # print('Speckle filter')
  spk = speckle_filter(TC)
  spk = deb

  print('Multilooking')
  mult = multilooking(spk)

  print('to dB')
  dB = linear_to_db(mult)
  write(dB, f'/media/jp/FreeAgent GoFlex Drive/SAR/SLC_raw/AE04/{area_name}_split_orb_cal_deb_spk_mult_TC_dB.tif')

In [19]:
workflow_SLC(file, 'area4')

read
TOPSAR-split
Orbit file

100% done.
OpenSearch: https://scihub.copernicus.eu/gnss/search?q=platformname:Sentinel-1 AND platformnumber:A AND producttype:AUX_POEORB AND beginposition:[2022-08-01T00:00:000Z TO 2022-08-31T24:00:000Z]
OpenSearch: https://scihub.copernicus.eu/gnss/search?q=platformname:Sentinel-1 AND platformnumber:A AND producttype:AUX_RESORB AND beginposition:[2022-08-01T00:00:000Z TO 2022-08-31T24:00:000Z]
No valid orbit file found for 17-AUG-2022 09:06:14.224466
Orbit files may be downloaded from https://scihub.copernicus.eu/gnss/odata/v1/
and placed in /home/jp/.snap/auxdata/Orbits/Sentinel-1/POEORB/S1A/2022/08


Orbit files may be downloaded from https://scihub.copernicus.eu/gnss/odata/v1/
and placed in /home/jp/.snap/auxdata/Orbits/Sentinel-1/POEORB/S1A/2022/08


RuntimeError: org.esa.snap.core.gpf.OperatorException: No valid orbit file found for 17-AUG-2022 09:06:14.224466
Orbit files may be downloaded from https://scihub.copernicus.eu/gnss/odata/v1/
and placed in /home/jp/.snap/auxdata/Orbits/Sentinel-1/POEORB/S1A/2022/08

INFO: org.hsqldb.persist.Logger: Database closed
